In [1]:
import sys, os, git, rstparse

In [2]:
from utilities import detect_block, block_to_sentence

In [3]:
current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/docs/inputs/shared-pyplot-files/")

In [4]:
tutorials = {"level0": ["lennard-jones-fluid"], }
             #"level1": ["breaking-a-carbon-nanotube"],
             #"level2": ["polymer-in-water", "nanosheared-electrolyte"],
             #"level3": ["water-adsorption-in-silica", "free-energy-calculation", "reactive-silicon-dioxide"]}

In [9]:
class RSTtoTEX:
    """Read RST file and convert it to TEX file."""
    def __init__(self, file_name, *args, **kwargs,):
        """Initialize"""
        super().__init__(*args, **kwargs)
        self.file_name = file_name

    def convert_file(self):
        """Main convert function."""
        self.read_rst()
        self.loop_through_file()

    def read_rst(self):
        """Convert the rst file into a list of strings"""
        rst = rstparse.Parser()
        with open(self.file_name) as f:
            rst.read(f)
        rst.parse()
        file_content = []
        for line in rst.lines:
            file_content.append(line)
        self.file_content = file_content

    def loop_through_file(self):
        """Loop over the rst file and extract the positions of items"""
        self.position_title = []
        self.position_labels = []
        self.position_hatnote = []
        self.position_paragraph = []
        for n, line in enumerate(self.file_content):
            self.detect_title_position(n, line)
            self.detect_label_position(n, line)
            self.detect_hatnote_position(n, line)
            self.detect_paragraph_position(n, line)
        self.detect_title()
        self.detect_label()
        self.detect_hatnote()
        self.detect_paragraph()

    def detect_title_position(self, n, line):
        if line[:3] == "***":
            self.position_title.append(n-1)

    def detect_title(self):
        if len(self.position_title) > 1:
            print("More than one title was found in", self.title)
        self.title = self.file_content[self.position_title[0]]

    def detect_label_position(self, n, line):
        if len(line.split()) == 2:
            if (line.split()[0] == '..') & (line.split()[1][0] == "_"):
                self.position_labels.append(n)

    def detect_label(self):
        if len(self.position_labels) > 1:
            print("More than one label was found in", self.title)
        self.label = self.file_content[self.position_labels[0]]

    def detect_hatnote_position(self, n, line):
        if len(line.split()) == 3:
            if (line.split()[0] == '..') & (line.split()[1][:-2] == "container") & (line.split()[2] == "hatnote"):
                self.position_hatnote.append(n)

    def detect_hatnote(self):
        if len(self.position_hatnote) > 1:
            print("More than one hatnote was found in", self.title)
        n = self.position_hatnote[0]
        line = detect_block(n+1, self.file_content)
        self.hatnote = block_to_sentence(line)[0]

    def detect_paragraph_position(self, n, line):
        if len(line.split()) == 3:
            if (line.split()[0] == '..') & (line.split()[1][:-2] == "container") & (line.split()[2] == "justify"):
                self.position_paragraph.append(n)

    def detect_paragraph(self):
        paragraphs = []
        for n in self.position_paragraph:
            paragraph = detect_block(n+1, self.file_content)
            paragraphs.append(block_to_sentence(paragraph))
        self.paragraphs = paragraphs

In [10]:
for level in tutorials.keys():
    for tutorial in tutorials[level]:
        rst_file_name = git_path+'/docs/sphinx/source/tutorials/'+level+'/'+tutorial+'.rst'
        RST = RSTtoTEX(file_name = rst_file_name)
        RST.convert_file()
    # print('\n')